 https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial/data?select=simpsons_dataset.csv

###Import relevant packages

In [ ]:
import pandas as pd

In [ ]:
from gensim.models import Word2Vec
import pandas as pd
import re
import numpy as np
from matplotlib import pyplot as plt

from sklearn.decomposition import PCA
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')
import gensim

In [ ]:
!pip install nltk

In [ ]:
import nltk

In [ ]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_data...
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Un


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
import gensim

In [ ]:
StopWords= stopwords.words("english")

In [ ]:
lemma = WordNetLemmatizer()

### Read the data and visualize

In [ ]:
data = pd.read_csv('/content/simpsons_dataset.csv')


In [ ]:
data.shape

(158314, 2)

In [ ]:
data.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


### Data cleaning

In [ ]:
data.isnull().sum()

raw_character_text    17814
spoken_words          26459
dtype: int64

In [ ]:
data = data.dropna().reset_index(drop=True)
data.isnull().sum()

raw_character_text    0
spoken_words          0
dtype: int64

In [ ]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
data["text_clean"]= data["spoken_words"].apply(lambda x: [i.lower() for i in x.split() if i not in StopWords])
data["text_clean"]= data["text_clean"].apply(lambda x: [i for i in x if i not in punctuation])
data["text_clean"]= data["text_clean"].apply(lambda x: [lemma.lemmatize(i) for i in x ])

In [ ]:
data.head()

,raw_character_text,spoken_words,text_clean
0,Miss Hoover,"No, actually, it was a little of both. Sometim...","[no,, actually,, little, both., sometimes, dis..."
1,Lisa Simpson,Where's Mr. Bergstrom?,"[where's, mr., bergstrom?]"
2,Miss Hoover,I don't know. Although I'd sure like to talk t...,"[i, know., although, i'd, sure, like, talk, hi..."
3,Lisa Simpson,That life is worth living.,"[that, life, worth, living.]"
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...,"[the, poll, open, end, recess., now,, case, de..."


### Apply Word2Vec model

In [ ]:
# train word2vec model
w2v = Word2Vec(data["text_clean"], min_count=5, vector_size = 10)
print(w2v)

Word2Vec<vocab=15409, vector_size=10, alpha=0.025>


In [ ]:
vocabulary= w2v.wv.index_to_key

In [ ]:
print(vocabulary)

['i', "i'm", 'you', 'like', 'and', 'get', 'oh,', 'the', 'well,', "that's", 'one', 'got', "it's", 'but', '--', 'know', "i'll", 'what', 'hey,', 'want', 'go', 'think', 'gonna', "can't", 'this', 'a', 'come', 'we', 'oh', 'make', 'take', 'look', "i've", 'little', 'could', 'it.', 'see', "we're", 'good', 'never', 'now', 'you.', 'no,', "he's", "you're", 'say', 'my', 'would', 'me.', 'yeah,', 'all', "there's", 'you,', "let's", 'it', 'need', 'back', 'time', 'uh,', 'that', 'going', "don't", 'love', 'mr.', 'really', 'u', 'thing', 'give', 'homer,', 'tell', 'so', 'let', 'if', "what's", 'now,', 'how', 'why', 'right', 'okay,', 'new', 'no', 'me,', 'two', 'something', 'it!', 'even', 'way', 'just', 'right,', 'bart,', 'he', 'marge,', 'thank', 'people', "i'd", "they're", 'maybe', 'ever', 'put', 'sure', 'man', 'old', 'here.', 'your', 'it,', 'made', 'first', "we'll", 'no.', 'still', 'yes,', 'well', 'is', 'stop', 'better', 'right.', 'big', 'homer', 'know,', 'always', 'they', 'day', 'call', 'dad,', 'gotta', 'not

In [ ]:
word_dict = {word:w2v.wv[word] for word in vocabulary}

In [ ]:
word_dict

{'i': array([ 0.02086545,  2.0811799 ,  1.0065722 ,  0.7574085 , -0.45509726,
        -0.92328197,  4.5634103 ,  2.3112373 ,  0.6686647 , -3.9808881 ],
       dtype=float32),
 "i'm": array([ 0.25553343,  0.35360628, -2.6587262 ,  2.523994  , -0.37647334,
        -1.23371   ,  1.7717704 ,  4.4869576 , -3.7966797 , -2.090783  ],
       dtype=float32),
 'you': array([ 1.101772  ,  2.457668  ,  2.1224408 ,  0.40336034, -1.2653841 ,
        -1.5912822 ,  3.1759872 ,  0.86387074, -0.36680302, -3.1296952 ],
       dtype=float32),
 'like': array([-1.0551821 ,  0.31675503,  2.9292946 , -2.8177433 ,  3.0210226 ,
        -1.5959086 ,  2.348181  , -0.39498737,  0.4040894 , -1.4687514 ],
       dtype=float32),
 'and': array([-1.7339953 ,  0.4661326 ,  0.1614189 , -0.7826587 ,  1.5824517 ,
         0.39061916,  2.116631  , -0.56935066, -0.12459628, -2.9592025 ],
       dtype=float32),
 'get': array([ 0.25642094,  1.2668198 , -0.753492  , -1.5715187 ,  3.094744  ,
         1.457412  ,  3.1573555 ,  0

In [ ]:
w2v.wv.most_similar("driving") #Words most similar to driving

[('using', 0.9922425746917725),
 ('freed', 0.9881362318992615),
 ('slept', 0.9872602820396423),
 ('invited', 0.9870587587356567),
 ('settled', 0.9857811331748962),
 ('swear,', 0.9852985739707947),
 ('works!', 0.9851364493370056),
 ('share,', 0.9850196838378906),
 ('fired', 0.9842123985290527),
 ('walked', 0.9835413694381714)]

In [ ]:
# summarize vocabulary
print(w2v.wv)
words1 = list(w2v.wv.key_to_index)
print(words1)

KeyedVectors<vector_size=10, 15409 keys>
['i', "i'm", 'you', 'like', 'and', 'get', 'oh,', 'the', 'well,', "that's", 'one', 'got', "it's", 'but', '--', 'know', "i'll", 'what', 'hey,', 'want', 'go', 'think', 'gonna', "can't", 'this', 'a', 'come', 'we', 'oh', 'make', 'take', 'look', "i've", 'little', 'could', 'it.', 'see', "we're", 'good', 'never', 'now', 'you.', 'no,', "he's", "you're", 'say', 'my', 'would', 'me.', 'yeah,', 'all', "there's", 'you,', "let's", 'it', 'need', 'back', 'time', 'uh,', 'that', 'going', "don't", 'love', 'mr.', 'really', 'u', 'thing', 'give', 'homer,', 'tell', 'so', 'let', 'if', "what's", 'now,', 'how', 'why', 'right', 'okay,', 'new', 'no', 'me,', 'two', 'something', 'it!', 'even', 'way', 'just', 'right,', 'bart,', 'he', 'marge,', 'thank', 'people', "i'd", "they're", 'maybe', 'ever', 'put', 'sure', 'man', 'old', 'here.', 'your', 'it,', 'made', 'first', "we'll", 'no.', 'still', 'yes,', 'well', 'is', 'stop', 'better', 'right.', 'big', 'homer', 'know,', 'always', 'th

In [ ]:
X = w2v.wv[w2v.wv.key_to_index]
print(X)
print("\n")

pca = PCA(n_components=2)
result = pca.fit_transform(X)
print(result)


[[ 0.02086545  2.0811799   1.0065722  ...  2.3112373   0.6686647
  -3.9808881 ]
 [ 0.25553343  0.35360628 -2.6587262  ...  4.4869576  -3.7966797
  -2.090783  ]
 [ 1.101772    2.457668    2.1224408  ...  0.86387074 -0.36680302
  -3.1296952 ]
 ...
 [ 0.08021289 -0.00584902  0.20049903 ...  0.01216549 -0.26591393
  -0.14467573]
 [ 0.02443143  0.02297668  0.14135574 ...  0.14861909 -0.3541363
  -0.03118827]
 [-0.01139082  0.02066358  0.1636721  ... -0.03906781 -0.08091713
  -0.14330833]]


[[ 3.053863   -2.349928  ]
 [ 1.8250331  -1.5027974 ]
 [ 2.2871583  -1.3813719 ]
 ...
 [-0.51393706 -0.14910252]
 [-0.52003247 -0.2551538 ]
 [-0.6446882  -0.13060278]]


### Apply CBOW model

In [ ]:
# Create CBOW model
model_cbow = gensim.models.Word2Vec(data["text_clean"], min_count = 5,
                              vector_size = 10, window = 5)

In [ ]:
cbow_similarity=model_cbow.wv.similarity('driving', 'bold')

### Apply Skipgram Model

In [ ]:
# Create Skip Gram model
model_skipgram = gensim.models.Word2Vec(data["text_clean"], min_count = 5, vector_size = 10,
                                             window = 5, sg = 1)

In [ ]:
skipgram_similarity=model_skipgram.wv.similarity('driving', 'bold')

###Compare both the models

In [ ]:
results=pd.DataFrame()

In [ ]:
results["Model"]=["CBOW", "SkipGram"]

In [ ]:
results["similarity"]= [cbow_similarity, skipgram_similarity]

In [ ]:
results

,Model,similarity
0,CBOW,0.923197
1,SkipGram,0.853895


Conclusion

In this implementation, we have applied word embeddings like word2vec using CBOW and Skipgram models.

For both the models, we have used the following hyper parameters.
vector_size= 10
window_size=5
min_count=5

Result table above shows the comparison